<a href="https://colab.research.google.com/github/leandrobarbieri/pydata-book/blob/2nd-edition/Group_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agrupamento e agregação
- 1: Separa os grupos, 2: Aplica funções de agregação, 3: Recombina em um novo df agregado.
- Cálculos estatísticos de resumo em colunas numéricas
- Iteração nos grupos
- Agrupamentos de colunas. Une várias colunas em uma
- Agregaçao com funções
- Agregaçao com mais de uma função por coluna numérica
- Agregação de cada coluna numerica com uma fução diferente
- Tabelas pivot e crosstab

In [632]:
# importações e configurações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)

# Group By

In [633]:
# cria um df de testes com dados categoricos (serão agrupado) e numéricos (serão agregados)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.arange(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0,-0.204708
1,a,two,1,0.478943
2,b,one,2,-0.519439
3,b,two,3,-0.555730
4,a,one,4,1.965781


In [634]:
# primeiramente os grupos são criados com base nas colunas passadas para groupby
# um obj do tipo pandas.core.groupby.generic.SeriesGroupBy é criado
# esse obj pode ser iterado
grouped = df.groupby(df['key1'])

for name, g in grouped:
  print("--------")
  print(name)
  print(g)

--------
a
  key1 key2  data1     data2
0    a  one      0 -0.204708
1    a  two      1  0.478943
4    a  one      4  1.965781
--------
b
  key1 key2  data1     data2
2    b  one      2 -0.519439
3    b  two      3 -0.555730


In [635]:
# Agregar (soma) a coluna 'data1' a partir do agrupamento da coluna 'key1'
soma_data1 = df['data1'].groupby(df['key1']).sum() 

# igual
# soma_data1 = df.groupby(['key1'])['data1'].sum()

soma_data1

key1
a    5
b    5
Name: data1, dtype: int64

In [636]:
# Agrupamento com múltiplas colunas (key1 e key2)
# Se não passar nada para df, todas as colunas numericas são agregadas
media_data1 = df['data1'].groupby([df['key1'], df['key2']]).mean()
media_data1

key1  key2
a     one     2
      two     1
b     one     2
      two     3
Name: data1, dtype: int64

In [637]:
# Outra forma de fazer o mesmo comando acima
# as_index=False faz com que as colunas agrupadas sejam tratadas como colunas e não como indices
df.groupby(['key1', 'key2'], as_index=False)[['data1']].count()

,key1,key2,data1
0,a,one,2
1,a,two,1
2,b,one,1
3,b,two,1


In [638]:
# usando unstack para pivotear o indice mais interno para colunas e criar uma matriz
media_data1.unstack()

key2,one,two
key1,,
a,2,1
b,2,3


In [639]:
# tamanho dos grupos, quantidade de linhas que foram agrupadas em cada grupo

# df.groupby(["key1", "key2"])[["data1", "data2"]].size()

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [640]:
# se não passar uma coluna numerica para df todas as colunas serão agregadas
# data1 e data2 são agregadas automaticamente
df.groupby(df["key1"]).sum()

# igual ao comando acima
# df.groupby("key1")[["data1", "data2"]].sum()

,data1,data2
key1,,
a,5,2.240016
b,5,-1.075169


## Iterando sob grupos
Antes de aplicar a função de agregação os grupos são separados em varios subgrupos de acordo com a combinação de colunas que foram usadas no groupby. Esses gupos podem ser acessados antes de aplicar a agregação.
> *o groupby devolve um objeto iteravel com uma tupla contendo o name do agrupamento um df com as linhas do grupo*

In [641]:
# Iterando nos grupos e acessadi a tupla (o primeiro elemento é a coluna usada para agrupar e o segundo são os registros)
# Fazer a iteração antes de aplicar a função de agregação
for nome, grupo in df.groupby(['key1', 'key2']):
  print(f"\nColunas agrupadas: {nome}")
  print(grupo)
  


Colunas agrupadas: ('a', 'one')
  key1 key2  data1     data2
0    a  one      0 -0.204708
4    a  one      4  1.965781

Colunas agrupadas: ('a', 'two')
  key1 key2  data1     data2
1    a  two      1  0.478943

Colunas agrupadas: ('b', 'one')
  key1 key2  data1     data2
2    b  one      2 -0.519439

Colunas agrupadas: ('b', 'two')
  key1 key2  data1    data2
3    b  two      3 -0.55573


In [642]:
# criando um dicionário onde a chave é uma tupla com as colunas agrupadas e os valores são as linhas dos subgrupos
dicionario_grupos = dict(list(df.groupby(['key1', 'key2'])))
dicionario_grupos

{('a', 'one'):   key1 key2  data1     data2
 0    a  one      0 -0.204708
 4    a  one      4  1.965781, ('a', 'two'):   key1 key2  data1     data2
 1    a  two      1  0.478943, ('b', 'one'):   key1 key2  data1     data2
 2    b  one      2 -0.519439, ('b', 'two'):   key1 key2  data1    data2
 3    b  two      3 -0.55573}

In [643]:
# acessando um subconjunto pela chave dos agrupamentos
dicionario_grupos[('a', 'one')]

,key1,key2,data1,data2
0,a,one,0,-0.204708
4,a,one,4,1.965781


In [644]:
# agrupando pelos tipos de colunas no eixo das colunas

# o df tem duas colunas float e duas object
print(f"Tipos de dados:\n{df.dtypes}\n")

# separar em 3 grupos as colunas de float, object, int. 
# axis=1 faz o agrupamento pelas colunas. Separa um grupo para cada coluna
grouped_columns = df.groupby(df.dtypes, axis=1)

# cada grupo apenas com colunas do tipo
# exemplo: 2 colunas do tipo obj estão no mesmo agrupamento
for tipo, dados in grouped_columns:
  print("----------")
  print(tipo)
  print(dados)

Tipos de dados:
key1      object
key2      object
data1      int64
data2    float64
dtype: object

----------
int64
   data1
0      0
1      1
2      2
3      3
4      4
----------
float64
      data2
0 -0.204708
1  0.478943
2 -0.519439
3 -0.555730
4  1.965781
----------
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [645]:
# Acessando o grupp de colunas do tipo obj
# Tranforma as tuplas em uma lista e acessa somente as colunas object
list(grouped_columns)[2]

(dtype('O'),   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one)

## Agrupando com dicts mapeados
> Passa um dicionário que determina a combinação de cada grupo, é como se estivessemos definindo um agrupamento sem que tenha um coluna no df, como um agrupamento externo usando um dict como referência para juntar linha ou colunas.

In [646]:
pessoas = pd.DataFrame(data=np.random.randn(5, 5),
                       columns=['a', 'b', 'c', 'd', 'e'],
                       index=["Leandro", "Bruna", "Breno", "João", "Maria"])
pessoas

,a,b,c,d,e
Leandro,1.393406,0.092908,0.281746,0.769023,1.246435
Bruna,1.007189,-1.296221,0.274992,0.228913,1.352917
Breno,0.886429,-2.001637,-0.371843,1.669025,-0.438570
João,-0.539741,0.476985,3.248944,-1.021228,-0.577087
Maria,0.124121,0.302614,0.523772,0.000940,1.343810


In [647]:
# será usado para agrupamento de colunas. Uma cor para mais de uma coluna
dict_mapeamento_grupos = {"a": "Red", "b": "Red", "c": "Red", "d": "Blue", "e": "Blue", "f":"Orange"}
dict_mapeamento_grupos

{'a': 'Red', 'b': 'Red', 'c': 'Red', 'd': 'Blue', 'e': 'Blue', 'f': 'Orange'}

In [648]:
# as colunas podem ser agrupadas de acordo com as combinações do dict
# cada chave do dict (a, b, c, d, e) é mapeada com cada coluna do df e é retornado o valor do dict como agrupamento
agrupamento_por_colunas = pessoas.groupby(dict_mapeamento_grupos, axis=1)

# aplica uma agregação no agrupamento. Por exemplo, soma a, b, c na cor Red
print(f"Quantidade de colunas de cada cor:\n{agrupamento_por_colunas.size()}\n")

# imprime os valores dos grupos antes da agregação. grupo[0] representa a chave de agrupamento e grupo[1] a dataframe do subgrupo
[print(f"Grupo: {nome}\n{grupo}\n") for nome, grupo in agrupamento_por_colunas]

agrupamento_por_colunas.sum()

Quantidade de colunas de cada cor:
Blue    2
Red     3
dtype: int64

Grupo: Blue
                d         e
Leandro  0.769023  1.246435
Bruna    0.228913  1.352917
Breno    1.669025 -0.438570
João    -1.021228 -0.577087
Maria    0.000940  1.343810

Grupo: Red
                a         b         c
Leandro  1.393406  0.092908  0.281746
Bruna    1.007189 -1.296221  0.274992
Breno    0.886429 -2.001637 -0.371843
João    -0.539741  0.476985  3.248944
Maria    0.124121  0.302614  0.523772



,Blue,Red
Leandro,2.015457,1.768060
Bruna,1.581830,-0.014040
Breno,1.230456,-1.487051
João,-1.598315,3.186187
Maria,1.344750,0.950507


In [649]:
# o mapeamento também funciona e pode ser feito com um obj series

# tranforma o dict em series
map_series = pd.Series(dict_mapeamento_grupos)

# utiliza uma series para fazer o mapeamento, para cada chave em series é mapeado com as colunas do df e retorna o valor da series como chave de agrupamento
pessoas.groupby(map_series, axis=1).sum()

,Blue,Red
Leandro,2.015457,1.768060
Bruna,1.581830,-0.014040
Breno,1.230456,-1.487051
João,-1.598315,3.186187
Maria,1.344750,0.950507


## Agrupamentos com funções

In [650]:
# agrupa pelo tamanho do nome da pessoa (como não passou nada usa o indice)
pessoas.groupby(len).sum()

,a,b,c,d,e
4,-0.539741,0.476985,3.248944,-1.021228,-0.577087
5,2.017740,-2.995245,0.426921,1.898878,2.258157
7,1.393406,0.092908,0.281746,0.769023,1.246435


In [651]:
# agrupando usando um nível de índice
indices_2niveis = pd.MultiIndex.from_arrays([["Leandro", "Bruna", "Breno", "João", "Maria"],
                                 ["natacao", "natacao", "natacao", "basquete", "atletismo"]], 
                                names=["familia", "esportes"])

# atribui o nome indice com 2 niveis para o dataframe
pessoas.index = indices_2niveis

# inverte os indices
pessoas.swaplevel("esportes", "familia")

a         b         c         d         e
esportes  familia                                                  
natacao   Leandro  1.393406  0.092908  0.281746  0.769023  1.246435
          Bruna    1.007189 -1.296221  0.274992  0.228913  1.352917
          Breno    0.886429 -2.001637 -0.371843  1.669025 -0.438570
basquete  João    -0.539741  0.476985  3.248944 -1.021228 -0.577087
atletismo Maria    0.124121  0.302614  0.523772  0.000940  1.343810

In [652]:
# agrupa pelo indice no level "esportes" apenas nas colunas numericas a e b
pessoas.groupby(level="esportes", axis=0)[["a", "b"]].sum()

,a,b
esportes,,
atletismo,0.124121,0.302614
basquete,-0.539741,0.476985
natacao,3.287025,-3.204951


# Agregação
Depois que os grupos são criados pela função groupby as funções de agregação entram em ação fazendo os calculos.
- count
- min, max
- str e var (desvio padrão e variancia)
- prod (produto dois valores)
- first, last

In [653]:
# soma
pessoas.reset_index()
pessoas.groupby(["esportes"])[["a", "b"]].sum()

,a,b
esportes,,
atletismo,0.124121,0.302614
basquete,-0.539741,0.476985
natacao,3.287025,-3.204951


In [654]:
# min
pessoas.groupby(["esportes"])[["a", "b"]].min()

,a,b
esportes,,
atletismo,0.124121,0.302614
basquete,-0.539741,0.476985
natacao,0.886429,-2.001637


In [655]:
# agg: usando suas proprias funções, ou uma lista de funções
# para cada coluna agregada, são aplicadas varias funções de agregação diferentes
pessoas.groupby("esportes").agg(["min", "max", "count", "sum"])[["a", "b"]]

a                  ...         b                
                min       max count  ...       max count       sum
esportes                             ...                          
atletismo  0.124121  0.124121     1  ...  0.302614     1  0.302614
basquete  -0.539741 -0.539741     1  ...  0.476985     1  0.476985
natacao    0.886429  1.393406     3  ...  0.092908     3 -3.204951

[3 rows x 8 columns]

In [656]:
def max_min_dif(dados):
  return dados.max() - dados.min()

# Agrupa os esportes e faz a agregação das colunas "a" e "b" aplicando as funções de agregação "max", "min", "max_min_dif" (customizada)
pessoas.groupby("esportes").agg(["count", "max", "min", max_min_dif])[["a", "b"]]

a                      ...         b                      
          count       max       min  ...       max       min max_min_dif
esportes                             ...                                
atletismo     1  0.124121  0.124121  ...  0.302614  0.302614    0.000000
basquete      1 -0.539741 -0.539741  ...  0.476985  0.476985    0.000000
natacao       3  1.393406  0.886429  ...  0.092908 -2.001637    2.094545

[3 rows x 8 columns]

In [657]:
pessoas.groupby("esportes").agg(["count", "mean", "std", "min", "quantile", "max"])[["a"]]

a                                                  
          count      mean       std       min  quantile       max
esportes                                                         
atletismo     1  0.124121       NaN  0.124121  0.124121  0.124121
basquete      1 -0.539741       NaN -0.539741 -0.539741 -0.539741
natacao       3  1.095675  0.264818  0.886429  1.007189  1.393406

In [658]:
# definindo o nome da função
# por padrão os nome das colunas recebem os nome das funções utilizadas
# para alterar os nomes passamos as tuplas na função agg
pessoas.groupby("esportes").agg([("Máximo", "max"), ("Mínimo", "min"), ("Quantidade", "count"), ("Mediana", "quantile")])["a"]

,Máximo,Mínimo,Quantidade,Mediana
esportes,,,,
atletismo,0.124121,0.124121,1,0.124121
basquete,-0.539741,-0.539741,1,-0.539741
natacao,1.393406,0.886429,3,1.007189


In [659]:
# todas as 3 funções de agregação são aplicadas as colunas a e b
func = ("count", "max", "min")
resultado = pessoas[["a", "b"]].groupby("esportes").agg(func)

resultado["a"]

,count,max,min
esportes,,,
atletismo,1,0.124121,0.124121
basquete,1,-0.539741,-0.539741
natacao,3,1.393406,0.886429


In [660]:
# retorna para o indice padrão
pessoas.reset_index(inplace=True)

# Para cada coluna numérica uma ou varias funções de agregação diferentes
# agrupa os esportes e agrega para a coluna "a" usando a função sum e a coluna "b" usando as funções count, max, min
pessoas.groupby("esportes", as_index=False).agg({"a": "sum", "b": ["count", "max", "min"]})

esportes         a     b                    
                   sum count       max       min
0  atletismo  0.124121     1  0.302614  0.302614
1   basquete -0.539741     1  0.476985  0.476985
2    natacao  3.287025     3  0.092908 -2.001637

# Apply
Chama uma função a cada parte separada em grupos e depois reagrupa

In [661]:
# top 5 por grupo
def topn_grupo(df, n=2, coluna="a"):
  
  # retorna os valores ordenados pela coluna e apenas os ultimos n [-n:]
  return df.sort_values(by=coluna)[-n:]

In [662]:
#todos os valores ordenados
pessoas.sort_values(by="a", ascending=False)[["esportes", "familia", "a"]]

,esportes,familia,a
0,natacao,Leandro,1.393406
1,natacao,Bruna,1.007189
2,natacao,Breno,0.886429
4,atletismo,Maria,0.124121
3,basquete,João,-0.539741


In [663]:
# top n para o df todo
topn_grupo(pessoas)

,familia,esportes,a,b,c,d,e
1,Bruna,natacao,1.007189,-1.296221,0.274992,0.228913,1.352917
0,Leandro,natacao,1.393406,0.092908,0.281746,0.769023,1.246435


In [664]:
# topn aplicado a cada grupo e depois os grupos são unidos usando a função concat (union) do pandas 
# neste exemplo, natação apesar de ter 3 linhas, retorna apenas os 2 primeiros
# os argumentos da função são passados depois da função (n=2 e coluna="a")
pessoas.groupby(["esportes"]).apply(topn_grupo, n=2, coluna="a")

familia   esportes         a  ...         c         d         e
esportes                                   ...                              
atletismo 4    Maria  atletismo  0.124121  ...  0.523772  0.000940  1.343810
basquete  3     João   basquete -0.539741  ...  3.248944 -1.021228 -0.577087
natacao   1    Bruna    natacao  1.007189  ...  0.274992  0.228913  1.352917
          0  Leandro    natacao  1.393406  ...  0.281746  0.769023  1.246435

[4 rows x 7 columns]

In [665]:
# os grupos por padrão viram indices
# para desativar group_keys=False
pessoas.groupby("esportes", group_keys=False).apply(topn_grupo, n=2, coluna="a")

,familia,esportes,a,b,c,d,e
4,Maria,atletismo,0.124121,0.302614,0.523772,0.000940,1.343810
3,João,basquete,-0.539741,0.476985,3.248944,-1.021228,-0.577087
1,Bruna,natacao,1.007189,-1.296221,0.274992,0.228913,1.352917
0,Leandro,natacao,1.393406,0.092908,0.281746,0.769023,1.246435


# Fillna com a média do grupo
Uma alternativa para o preenchimento de valores NaN é remover, usar a média da coluna ou zerar valor. Mas pode ser interessante atribuir a média do valor de acordo com um agrupamento significativo no qual o registro pertence.

In [666]:
# criando uma serie com alguns elementos NaN
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

data

Ohio         -0.713544
New York     -0.831154
Vermont            NaN
Florida      -1.860761
Oregon       -0.860757
Nevada             NaN
California   -1.265934
Idaho              NaN
dtype: float64

In [667]:
# criando uma lista para ser usada como agrupamento
group_key = ['East'] * 4 + ['West'] * 4

# média dos grupos
data.groupby(group_key).mean()

East   -1.135153
West   -1.063346
dtype: float64

In [668]:
# função para preencher com a média do grupo passado como param
fill_mean = lambda g: g.fillna(g.mean())

# separa os grupos por "east" e "west", aplica a função lambda, passa o grupos para a função que preenche com fillna
data.groupby(group_key).apply(fill_mean)

Ohio         -0.713544
New York     -0.831154
Vermont      -1.135153
Florida      -1.860761
Oregon       -0.860757
Nevada       -1.063346
California   -1.265934
Idaho        -1.063346
dtype: float64

In [669]:
# preenchimento dos grupos com valores fixos
fill_values = {'East': 999, 'West': -999}

# usa o atributo name ("east" e "west") interno de cada grupo para buscar no dict a chave e retorna o valor fixo para fillna
fill_func = lambda g: g.fillna(fill_values[g.name])

# separa os grupos por "east" e "west", aplica a função lambda
data.groupby(group_key).apply(fill_func)

# valoes name e g do grupo
# for name, g in data.groupby(group_key):
#   print(name)
#   print(g)

Ohio           -0.713544
New York       -0.831154
Vermont       999.000000
Florida        -1.860761
Oregon         -0.860757
Nevada       -999.000000
California     -1.265934
Idaho        -999.000000
dtype: float64

# Média ponderada de grupos
media ponderada: soma da multiplicação dos valores pelos pesos, dividido pela soma dos pesos

In [670]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

df["calc"] = df["data"] * df["weights"]
df

,category,data,weights,calc
0,a,-1.063512,0.125842,-0.133834
1,a,0.332883,0.687596,0.228889
2,a,-2.359419,0.799607,-1.886607
3,a,-0.199543,0.573537,-0.114445
4,b,-1.541996,0.973230,-1.500716
5,b,-0.970736,0.634054,-0.615499
6,b,-1.307030,0.888422,-1.161194
7,b,0.286350,0.495415,0.141862


In [671]:
# media ponderada
df["calc"].sum() / df["weights"].sum()

-0.9737033772826028

In [672]:
# separa em grupos "a" e "b"
grouped = df.groupby('category')

# recebe o grupo e faz o calculo da média ponderada
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

# passa os grupos e aplica a média ponderada em cada grupos e depois concatena (union)
grouped.apply(get_wavg)

category
a   -0.871680
b   -1.048285
dtype: float64

# Pivot table
Uma alternativa conveniente para agrupamentos e agregações com recursos para calculos de subtotais

In [673]:
pessoas

,familia,esportes,a,b,c,d,e
0,Leandro,natacao,1.393406,0.092908,0.281746,0.769023,1.246435
1,Bruna,natacao,1.007189,-1.296221,0.274992,0.228913,1.352917
2,Breno,natacao,0.886429,-2.001637,-0.371843,1.669025,-0.438570
3,João,basquete,-0.539741,0.476985,3.248944,-1.021228,-0.577087
4,Maria,atletismo,0.124121,0.302614,0.523772,0.000940,1.343810


In [674]:
# a agregação padrão usada é aggfunc=mean caso não seja informado
pessoas.pivot_table(index=["esportes", "familia"])

a         b         c         d         e
esportes  familia                                                  
atletismo Maria    0.124121  0.302614  0.523772  0.000940  1.343810
basquete  João    -0.539741  0.476985  3.248944 -1.021228 -0.577087
natacao   Breno    0.886429 -2.001637 -0.371843  1.669025 -0.438570
          Bruna    1.007189 -1.296221  0.274992  0.228913  1.352917
          Leandro  1.393406  0.092908  0.281746  0.769023  1.246435

In [675]:
# agrega esportes nas colunas, familia nas linhas e usa apenas as medidas "a" e "b" com sum para agregação
# values=<colunas usadas para agregar os valores "medidas"> 
# index=<atributo usado nas linhas>
# columns=<atributo usado nas colunas>
# aggfunc=<função de agregação>
# margins=<true para incluir subtotais>
# fill_value=<preenche as combinações vazias com o valor informado>
pessoas.pivot_table(values=["a", "b"], index=["familia"], columns=["esportes"], aggfunc=sum, margins=True, margins_name="Total Parcial", fill_value="-")

a                      ...         b                         
esportes      atletismo  basquete   natacao  ...  basquete    natacao Total Parcial
familia                                      ...                                   
Breno                 -         -  0.886429  ...         -   -2.00164      -2.00164
Bruna                 -         -   1.00719  ...         -   -1.29622      -1.29622
João                  - -0.539741         -  ...  0.476985          -      0.476985
Leandro               -         -   1.39341  ...         -  0.0929079     0.0929079
Maria          0.124121         -         -  ...         -          -      0.302614
Total Parcial  0.124121 -0.539741   3.28702  ...  0.476985   -3.20495      -2.42535

[6 rows x 8 columns]

# Crosstab
É um caso especial de aplicação pivot_table

In [676]:
# transforma uma string em um stream de string para leitura
from io import StringIO

data = """\
Medalhas  Nationality  Handedness
10   USA  Right-handed
20   Japan    Left-handed
13   USA  Right-handed
4   Japan    Right-handed
15   Japan    Left-handed
6   Japan    Right-handed
17   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

# cria um df lendo um StringIO com separador de espaços
# sep='s+' representa serador de espaços com quantidade de espaços variado (separa as colunas da tabela quando identifica um ou mais espaços)
data = pd.read_table(StringIO(data), sep='\s+')

data.sort_values(by="Nationality")

,Medalhas,Nationality,Handedness
1,20,Japan,Left-handed
3,4,Japan,Right-handed
4,15,Japan,Left-handed
5,6,Japan,Right-handed
8,9,Japan,Right-handed
0,10,USA,Right-handed
2,13,USA,Right-handed
6,17,USA,Right-handed
7,8,USA,Left-handed
9,10,USA,Right-handed


In [677]:
# por padrão a função de agregação é a quantidade de registros
# margins True conta a quantidade de registros
pd.crosstab(index=data["Nationality"], columns=data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [678]:
# por padrão a função de agregação é a quantidade de registros
# para alterar a função de agregação passamos values= e a função aggfunc
pd.crosstab(index=data["Nationality"], columns=data["Handedness"], margins=True, values=data["Medalhas"], aggfunc="sum")

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,35,19,54
USA,8,50,58
All,43,69,112


In [679]:
# normalize=True transforma os numeros e percentual do total geral
pd.crosstab(index=data["Nationality"], columns=data["Handedness"], margins=True, values=data["Medalhas"], aggfunc="sum", normalize=True, margins_name="Total")

Handedness,Left-handed,Right-handed,Total
Nationality,,,
Japan,0.312500,0.169643,0.482143
USA,0.071429,0.446429,0.517857
Total,0.383929,0.616071,1.000000
